In [2]:
import keras

import os
import sys
import operator
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import hashlib
import re
from collections import defaultdict

from keras.layers import Merge
from keras.models import Sequential
from keras.layers import Dense, Activation,Lambda
from keras.optimizers import SGD,Adam
from keras.utils import np_utils
from keras.layers import Input

Using Theano backend.
Using gpu device 0: GeForce GTX 780 Ti (CNMeM is disabled, cuDNN 5105)
/home/jianqi/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [5]:
data_path = "../input/"
train_file = data_path + "train.json"
test_file = data_path + "test.json"
train_df = pd.read_json(train_file)
test_df = pd.read_json(test_file)
original_col = train_df.columns

colnames = ['features','latitude','longitude','price','bathrooms','bedrooms','interest_level']

df = train_df[colnames].copy()

df['features'] = df['features'].map(lambda x:[tt.lower() for tt in x])
#df['description'] = df['description'].map(lambda x:x.lower())
    
all_fea = defaultdict(int)
for _,row in df.iterrows():
    for xx in row['features']:
        all_fea[xx] += 1



In [7]:
for _,row in test_df.iterrows():
    for xx in row['features']:
        if 'laundary' in xx.lower():
            print row['features']

In [32]:
fea_list = [k for (k,v) in all_fea.iteritems() if v>=3]

fea_map = {fea_list[ii]:ii for ii in range(len(fea_list))}

df = df.reset_index(drop=True)
text_fea = np.zeros((len(df),len(fea_map)))
for inx,row in df.iterrows():
    for xx in row['features']:
        if xx in fea_map:
            text_fea[inx][fea_map[xx]] = 1

(49352, 355)

In [33]:
quant_fea = df[['latitude','longitude','price','bathrooms','bedrooms']].as_matrix()

In [34]:
quant_fea.shape

(49352, 5)

In [35]:
target_num_map = {'high':0, 'medium':1, 'low':2}
y = np.array(df['interest_level'].apply(lambda x: target_num_map[x]))
y = np_utils.to_categorical(y, nb_classes=3)

In [36]:
y.shape

(49352, 3)

In [37]:
model = Sequential()

In [38]:
left_branch = Sequential()
left_branch.add(Dense(120, input_dim=355))

#right_branch = Sequential()
#right_branch = Input(shape=(5,))

right_branch = Sequential()
right_branch.add(Lambda(lambda x: x, input_shape=(quant_,)))

merged = Merge([left_branch, right_branch], mode='concat')

final_model = Sequential()
final_model.add(merged)
final_model.add(Dense(16,activation='relu'))
final_model.add(Dense(3, activation='softmax'))

AttributeError: 'TensorVariable' object has no attribute 'get_output_shape_at'

In [3]:
compress_layer = Dense(120, input_dim=455,activation='linear',bias=False)

In [4]:
dir(compress_layer)

['W_constraint',
 'W_regularizer',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_node_attribute_at_index',
 '_non_trainable_weights',
 '_trainable_weights',
 'activation',
 'activity_regularizer',
 'add_inbound_node',
 'assert_input_compatibility',
 'b_constraint',
 'b_regularizer',
 'batch_input_shape',
 'bias',
 'build',
 'built',
 'call',
 'compute_mask',
 'constraints',
 'count_params',
 'create_input_layer',
 'from_config',
 'get_config',
 'get_input_at',
 'get_input_mask_at',
 'get_input_shape_at',
 'get_output_at',
 'get_output_mask_at',
 'get_output_shape_at',
 'get_output_shape_for',
 'get_weights',
 'inbound_nodes',
 'init',
 'initial_weights',
 'input',
 'input_dim',
 'input_dtype',
 'input_mask',
 'input_shape',
 'input_spec',
 'name'

In [43]:
final_model.compile(optimizer='Adam', loss='categorical_crossentropy')
final_model.fit([text_fea, quant_fea], y)  # we pass one data array per model input

Exception: Error when checking model target: expected dense_5 to have shape (None, 3) but got array with shape (49352, 1)